<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morfologiczna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy

     |████████████████████████████████| 6.0 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 59.4 MB/s 
     |████████████████████████████████| 451 kB 60.1 MB/s 
     |████████████████████████████████| 10.1 MB 60.8 MB/s 
     |████████████████████████████████| 628 kB 60.0 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from gensim.utils import simple_preprocess
from collections import Counter
import gc

import spacy
from spacy.lang.pl.stop_words import STOP_WORDS

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')

In [4]:
# Wyświetlanie szerszych kolumn, aby nie skracać wyświetlanych nazw tematów
pd.set_option('max_colwidth', 400)

In [5]:
!python -m spacy download pl_core_news_lg

     |████████████████████████████████| 612.1 MB 11 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')


In [6]:
nlp = spacy.load('pl_core_news_lg')

# Przygotowanie danych

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_excel('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/budzet_obywatelski.xlsx', sheet_name='Arkusz1')

In [9]:
df.rename(columns={'rok ': 'rok', 'miasto ':'miasto'}, inplace=True)

In [10]:
# Usunięcie wartości pustych
df = df[~df['nazwa'].isna()]

In [11]:
df['miasto'] = df['miasto'].map(lambda x: x.upper())

In [12]:
# Usunięcie pustych wpisów

# Indeksy do usunięcia
rem_index = df[df['nazwa'].isnull()].index.tolist()
rem_index += df[df['nazwa']=='bd'].index.tolist()

df.drop(df.index[rem_index], inplace=True)

In [13]:
# Usunięcie duplikatów
df = df.drop_duplicates()

In [14]:
df['nazwa'] = df['nazwa'].apply(str)

In [15]:
#usuniecie kolumny wszystko
if 'WSZYSTKO' in df: del df['WSZYSTKO']

In [16]:
df.sample(10)

,rok,miasto,nazwa
686,2018,GDYNIA,Rozbudowa placu zabaw dla dzieci matych i wiekszych wraz Z miejscem do wypoczynku miedzy ul. Morska 107-109 a Komandorska 18-20
3441,2020,JAWORZNO,"""Jaworznickie Przystanki Rowerowe w GEOsferze, Parku Gródek i Centrum wraz z samoobsługowym punktem naprawy rowerów"" - szacunkowe koszty: 98 400,00 zł"
5453,2020,TYCHY,Nasadzenia drzew przy ul. Asnyka
11043,2018,BIAŁA PODLASKA,"Rowerowy plac zabaw ""PUMPTRACK"" - kontynuacja budowy SKATEPARKU"
12282,2018/19,ŁÓDŹ,Robotyka w szkole.
2567,2020,CHORZÓW,Festyn rodzinny: 90 lat Maciejkowic w Chorzowie
6655,2019,KALISZ,Utworzenie centrum badawczego i eksperymentalnego w Publicznym Przedszkolu Nr 27 „Radość”.
5114,2018,SOSNOWIEC,Renowacja schodów pomiedzy ul. Sniezna a ul. Bedzinska
15304,2020,ŁÓDŹ,Nowy wóz ratowniczo-gaśniczy gwarancją poprawy waszego bezpieczeństwa.
12753,2018/19,ŁÓDŹ,"Sąsiedzki skwer Piotrkowska 257 - ławki dla przechodniów, nasadzenia zieleni oraz kosze na odchody."


In [ ]:
df.loc[(df['miasto'] == 'WARSZAWA'), "czy_WAW"] = 1
df.loc[df["czy_WAW"].isnull(),  "czy_WAW"] = 0

In [ ]:
df["czy_WAW"].value_counts()

0.0    18431
1.0     4599
Name: czy_WAW, dtype: int64

# Warszawa

In [ ]:
df_waw = df[df['miasto'] == 'WARSZAWA']
df_waw.head()

,rok,miasto,nazwa,czy_WAW
18418,2018,WARSZAWA,Domki dla jeży,1.0
18419,2018,WARSZAWA,Wolskie lato filmowe,1.0
18420,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki lęgowe,1.0
18421,2018,WARSZAWA,Czysta Wisła na Pradze Południe,1.0
18422,2018,WARSZAWA,Wola na 2 koła! Kontraruch rowerowy w całej dzielnicy,1.0


In [ ]:
#zlaczenie wszystkich nazw w jeden string
docs_waw = ' '.join(df_waw.nazwa)
docs_waw = docs_waw.lower()

In [ ]:
docs_waw

'domki dla jeży wolskie lato filmowe chronimy jerzyki i wróble na woli - skrzynki lęgowe czysta wisła na pradze południe wola na 2 koła! kontraruch rowerowy w całej dzielnicy bielańska plaża nad wisłą chronimy jerzyki i wróble na pradze -południe - skrzynki lęgowe leczenie i sterylizacja wolno żyjących kotów ćwiczenia dla seniorów na otwartych siłowniach rozwijamy wolskie maluchy skarby woli. czego nie wiesz o swojej dzielnicy? chronimy jerzyki i wróble na bielanach - skrzynki lęgowe czyste powietrze dla targówka - nasadzenia drzew tai chi na świeżym powietrzu ogródek przy przystanku autobusowym "antka rozpylacza 01" wiem czym oddycham! - tablica informująca o poziomie zanieczyszczenia powietrza (praga-południe) virtualna warszawa – turystyka dostępna dla wszystkich, również niepełnosprawnych. ratujemy drzewa parku skaryszewskiego - wiązania elastyczne w koronach drzew na podstawie ekspertyzy dendrologicznej. serek - miejsce spotkań i relaksu przy metrze słodowiec wodny plac zabaw dla 

In [ ]:
doc=nlp(docs_waw)

In [ ]:
attribs = ['orth_', 'lemma_', 'tag_', 'pos_', 'dep_', 'head', 'is_stop', 'is_punct', 'is_digit']
table = [{att:tok.__getattribute__(att) for att in attribs} for tok in doc]
df_waw_morf = pd.DataFrame(table)

In [ ]:
df_waw_morf.sample(50)

,orth_,lemma_,tag_,pos_,dep_,head,is_stop,is_punct,is_digit
13062,kaskada,kaskada,SUBST,NOUN,nmod,łąka,False,False,False
3275,poznania,poznanie,GER,NOUN,nmod:arg,sala,False,False,False
28698,przy,przy,PREP,ADP,case,przejazdach,True,False,False
1486,zakaz,zakaz,SUBST,NOUN,ROOT,zakaz,False,False,False
112,-,-,SUBST,PUNCT,punct,południe,False,True,False
24886,roboty,robota,SUBST,NOUN,conj,origami,False,False,False
2278,\r\n\t\t,\r\n\t\t,_SP,SPACE,dep,hotel,False,False,False
39710,na,na,PREP,ADP,case,odolanach,True,False,False
31711,przyjaciela,przyjaciel,SUBST,NOUN,conj,partnera,False,False,False
36085,sprzątania,sprzątanie,GER,NOUN,conj,ciała,False,False,False


In [ ]:
df_waw_morf.shape

(40358, 9)

In [ ]:
# Usunięcie duplikatów
df_waw_morf = df_waw_morf.drop_duplicates()

In [ ]:
df_waw_morf.shape

(40161, 9)

_pos: <br>
ADJ adjective *big, old, green, incomprehensible, first*<br>
ADP adposition *in, to, during*<br>
ADV adverb *very, tomorrow, down, where, there*<br>
AUX auxiliary *is, has (done), will (do), should (do)*<br>
CONJ conjunction *and, or, but*<br>
CCONJ coordinating conjunction *and, or, but*<br>
DET determiner *a, an, the*<br>
INTJ interjection *psst, ouch, bravo, hello*<br>
NOUN noun *girl, cat, tree, air, beauty*<br>
NUM numeral *1, 2017, one, seventy-seven, IV, MMXIV*<br>
PART particle *’s, not,*<br>
PRON pronoun *I, you, he, she, myself, themselves, somebody*<br>
PROPN proper noun *Mary, John, London, NATO, HBO*<br>
PUNCT punctuation *., (, ), ?*<br>
SCONJ subordinating conjunction *if, while, that*<br>
SYM symbol *$, %, §, ©, +, −, ×, ÷, =, :), *<br>
VERB verb *run, runs, running, eat, ate, eating*<br>
X other *sfpksdpsxmsa*<br>
SPACE space





In [ ]:
df_waw_morf.pos_.value_counts()

NOUN     16011
ADJ       6793
ADP       5674
PUNCT     4830
CCONJ     1514
X         1472
PROPN     1146
VERB       768
SPACE      544
ADV        398
NUM        363
DET        229
PRON       209
PART        76
AUX         66
SCONJ       41
SYM         17
INTJ        10
Name: pos_, dtype: int64

In [ ]:
df_waw_morf[ df_waw_morf['pos_']=='X'].sample(5)

,orth_,lemma_,tag_,pos_,dep_,head,is_stop,is_punct,is_digit
9904,c.d,c.d,SUBST,X,nmod,rewitalizacja,False,False,False
1657,nr,nr,BREV,X,nmod,dorosłych,False,False,False
6372,j,j,BREV,X,nmod,ul,False,False,False
36932,ul,ul,BREV,X,nmod,budowy,False,False,False
21701,nr,nr,BREV,X,nmod,bibliotece,False,False,False


In [ ]:
df_waw_morf.tag_.value_counts()

SUBST      17797
ADJ         6422
PREP        5681
INTERP      4349
CONJ        1516
BREV         856
NUM          758
GER          598
_SP          544
FIN          325
ADV          299
IMPT         224
PACT         170
PPAS         161
QUB          138
INF           97
PPRON12       38
ADJA          37
PPRON3        32
PRED          27
PRAET         24
XXX           18
SIEBIE        15
COMP           9
ADJP           7
DEPR           6
PCON           4
IMPS           4
BEDZIE         2
INTERJ         2
ADJC           1
Name: tag_, dtype: int64

In [ ]:
df_waw_morf.dep_.value_counts()

nmod                9099
amod                6048
case                5672
punct               4884
conj                3662
nmod:arg            2248
cc                  1495
obj                 1290
nsubj                754
obl                  707
ROOT                 626
dep                  566
appos                420
acl                  380
flat                 237
advmod               227
nummod               210
obl:arg              206
fixed                173
iobj                 170
parataxis:insert     138
amod:flat            128
nmod:flat            104
nummod:gov            86
det:poss              85
expl:pv               59
det                   57
cop                   57
det:numgov            51
parataxis:obj         50
advmod:neg            44
mark                  37
xcomp                 34
advmod:emph           30
acl:relcl             18
advcl                 14
xcomp:pred            10
obl:agent              9
obl:cmpr               8
cc:preconj             8


In [ ]:
df_waw_morf_verb = df_waw_morf[ df_waw_morf['pos_']=='VERB']

In [ ]:
df_waw_morf_verb.shape

(768, 9)

In [ ]:
df_waw_morf_verb["lemma_"].value_counts().head(60)

chronić        33
pić            31
ratować        26
przejść        16
wiedzieć       15
chcieć         14
mieć           14
grać           14
kochać         12
poznać         11
dbać            9
sadzić          8
czytać          8
łączyć          7
tańczyć         7
biegać          7
spotkać         7
wrzeciono       7
uczyć           7
ćwiczyć         6
zazieleniać     6
rozwijać        6
walczyć         6
bawić           6
kasprowicza     5
wspierać        5
integrować      5
zadbać          5
jeść            5
stworzyć        5
sprzątać        5
parkować        5
bruna           5
móc             4
wybierać        4
zapobiegać      4
zapraszać       4
płacić          4
oddychać        4
mieść           4
potrzebować     4
żoliborza       4
marnować        4
stać            4
być             4
mieszkać        4
zazielenić      4
rozumieć        4
uratować        3
posprzątać      3
reagować        3
przywracać      3
zobaczyć        3
pamiętać        3
zrobić          3
wzmocnić  

In [ ]:
df_waw_morf[ df_waw_morf['pos_']=='NOUN']["lemma_"].value_counts().head(60)

dziecko           406
park              279
plac              262
zabawa            257
ulica             240
biblioteka        231
chodnik           223
warsztat          220
zajęcie           188
szkoła            170
drzewo            159
remonta           152
młodzież          122
skwer             120
kultura           119
pieszy            118
mieszkaniec       114
zieleń            111
spotkanie         105
zakup             103
miejsce           102
dzielnica         101
boisko             99
senior             95
rewitalizacja      90
oświetlenie        89
gra                88
książka            83
budowa             81
dorosły            79
przejście          78
podwórko           77
siłownia           75
krzew              75
ławka              74
modernizacja       72
ogród              67
dom                63
droga              62
teren              60
cykl               60
rower              59
nauka              59
nawierzchnia       57
wypożyczalnia      53
przystanek

In [ ]:
df_waw_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Spacy_WAW.csv', index=False)

# Reszta miast

In [ ]:
df_roc = df[df['miasto'] != "WARSZAWA"]
df_roc.head(10)

,rok,miasto,nazwa,czy_WAW
0,2020,GDYNIA,Projektistatus,0.0
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp.",0.0
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy Obłuże rozwijające pamięć i koncentrację,0.0
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",0.0
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy ul. Niskiej na odcinku pomiędzy ul. Bohaterów Getta Warszawskiego i Hodowlaną,0.0
5,2020,GDYNIA,Cotygodniowe weekendowe lekcje tańca towarzyskiego dla dorosłych,0.0
6,2020,GDYNIA,Meble miejskie zaprojektowane przez dzieci. Podziemia Hal Targowych w Gdyni.,0.0
7,2020,GDYNIA,"Odnowienie podwórek w obrębie ulic Płk. Dąbka, Kwiatkowskiego, Sucharskiego, Ledóchowskiego i Maciejewicza",0.0
8,2020,GDYNIA,Poprawa atrakcyjności skweru przy skrzyżowaniu Wielkopolska-Łowicka-Płocka przez nasadzenie roślin ozdobnych,0.0
9,2020,GDYNIA,Radarowe wyświetlacze prędkości na ul. Morskiej,0.0


In [22]:
#funkcja zliczajaca wszystkie nazwy w jeden string i tworzaca tabele morf
def create_morf_table(df):
  docs_roc = ' '.join(df.nazwa)
  docs_roc = docs_roc.lower()
  doc=nlp(docs_roc)
  attribs = ['orth_', 'lemma_', 'tag_', 'pos_', 'dep_', 'head', 'is_stop', 'is_punct', 'is_digit']
  table = [{att:tok.__getattribute__(att) for att in attribs} for tok in doc]
  morf_table = pd.DataFrame(table)
  return morf_table

In [ ]:
nlp.max_length = 1500000

In [ ]:
df_roc_morf = create_morf_table(df_roc)

In [ ]:
df_roc_morf.sample(20)

,orth_,lemma_,tag_,pos_,dep_,head,is_stop,is_punct,is_digit
193614,i,i,CONJ,CCONJ,cc,młodzieży,True,False,False
142224,mieszkańców,mieszkaniec,SUBST,NOUN,obl,przyjazny,False,False,False
112569,zielone,zielony,ADJ,ADJ,amod,skwery,False,False,False
165151,ulicy,ulica,SUBST,NOUN,nmod,budowa,False,False,False
194861,podłoża,podłoże,SUBST,NOUN,nmod:arg,budowa,False,False,False
42139,budowa,budowa,SUBST,NOUN,conj,zakup,False,False,False
91905,raczka,raczek,SUBST,NOUN,nmod,zlota,False,False,False
34665,jasne,jasny,SUBST,ADJ,amod,czestochowie,False,False,False
179362,o,o,PREP,ADP,case,zalewie,True,False,False
15726,70,70,ADJ,NUM,dep,ul,False,False,True


In [ ]:
df_roc_morf.shape

(202507, 9)

In [ ]:
# Usunięcie duplikatów
df_waw_morf = df_waw_morf.drop_duplicates()

In [ ]:
df_roc_morf.shape

(202507, 9)

In [ ]:
df_waw_morf.pos_.value_counts()

NOUN     16011
ADJ       6793
ADP       5674
PUNCT     4830
CCONJ     1514
X         1472
PROPN     1146
VERB       768
SPACE      544
ADV        398
NUM        363
DET        229
PRON       209
PART        76
AUX         66
SCONJ       41
SYM         17
INTJ        10
Name: pos_, dtype: int64

In [ ]:
df_roc_morf[ df_roc_morf['pos_']=='VERB']["lemma_"].value_counts().head(60)

mieć            94
grać            67
ratować         51
bawić           42
uczyć           40
działać         39
łączyć          38
chcieć          33
czytać          33
pomóc           32
przejść         27
stworzyć        24
mieszkać        24
zadbać          23
dać             22
pomagać         21
być             21
dbać            17
poznać          17
móc             17
ćwiczyć         16
wychowywać      15
żyć             14
zapraszać       14
sadzić          14
powstać         13
wyremontować    13
zmieniać        13
tworzyć         12
chodzić         12
zajeó           11
biegać          11
obejmować       11
kasprowicza     11
prusa           11
zrobić          11
staszica        11
wspierać        11
uratować        10
zakładać        10
przywrócić       9
stać             9
lubić            9
wygon            9
dobrzec          8
pozwolić         8
zwolnić          8
kochać           8
janke            8
walczyć          8
spotkać          8
‐                8
henryka     

In [ ]:
df_roc_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Spacy_ROC.csv', index=False)

## ROK 2018

In [24]:
df_2018 = df[df['rok'] == 2018]
df_2018.head(10)

,rok,miasto,nazwa
654,2018,GDYNIA,Urzadzenie zieleni miejskiej miedzy ul. Dedala 6 a korona klifu
655,2018,GDYNIA,"Zagospodarowanie terenu po bytym baraku przy ulicy Dedala - stworzenie sciezek, ustawienie fawek, nasadzenia zieleni i aranzacja matej architektury"
656,2018,GDYNIA,Montaz fontann wody pitnej W ruchliwych punktach dzielnicy: przy placu zabaw oraz na poczatku sciezki rowerowej
657,2018,GDYNIA,"Rozbudowa placu zabaw dla dzieci mtodszych, starszych i mtodziezy przy ulicy Dedala 6-8"
658,2018,GDYNIA,Montaz dwóch defibrylatorów - przy parafii przy ul. Rybaków oraz przy wspólnocie mieszkaniowej przy ul. Ikara 6
659,2018,GDYNIA,"Nasadzenia kwiatów i innych kompozycji roslinnych przy ulicach Dedala, Ikara, Zielonej i Rybaków"
660,2018,GDYNIA,Oswietlenie boiska przy Szkole Podstawowej nr 28
661,2018,GDYNIA,Wykonanie ogólnodostepnego parkingu na terenie parafii Biskupa Józefa Sebastiana Pelczara
662,2018,GDYNIA,"Budowa rekreacyjnych torów do jazdy typu pumptrack wraz Z uzupelniajaca zielenia i miejscami do odpoczynku W poblizu przystanku autobusowego ""Gierdziejewskiego"""
663,2018,GDYNIA,"Wymiana oswietlenia przy gtównych ulicach dzielnicy na oprawy LED oraz montaz solarnych tawek (ul. Okreina, Chwarznienska, Wiczlinska, Zaruskiego, Filipkowskiego i Staniszewskiego)"


In [28]:
df_2018_morf = create_morf_table(df_2018)
df_2018_morf.head()

,orth_,lemma_,tag_,pos_,dep_,head,is_stop,is_punct,is_digit
0,urzadzenie,urzadzenie,GER,NOUN,nsubj,zagospodarowanie,False,False,False
1,zieleni,zieleń,SUBST,NOUN,obj,urzadzenie,False,False,False
2,miejskiej,miejski,ADJ,ADJ,amod,zieleni,False,False,False
3,miedzy,miedzy,PREP,ADP,case,ul,True,False,False
4,ul,ul,BREV,X,nmod,urzadzenie,False,False,False


In [30]:
df_2018_morf.shape

(58495, 9)

In [32]:
# Usunięcie duplikatów
df_2018_morf = df_2018_morf.drop_duplicates()
df_2018_morf.shape

(58111, 9)

In [35]:
df_2018_morf[ df_2018_morf['pos_']=='VERB']["lemma_"].value_counts().head(30)

pić            31
łączyć         17
chronić        16
grać           14
mieć           13
ratować        13
wiedzieć       11
mieszkać       10
poznać          9
chcieć          9
przejść         9
czytać          9
być             9
bawić           8
uczyć           6
‐               6
zadbać          6
potrzebować     5
biegać          5
zrobić          5
pomagać         5
ćwiczyć         5
oddychać        5
żyć             5
zamienić        4
dać             4
dobrzec         4
zapraszać       4
zajeó           4
zagrać          4
Name: lemma_, dtype: int64

In [36]:
df_2018_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Spacy_2018.csv', index=False)

## 2019

In [38]:
df_2019 = df[df['rok'] == 2019]
df_2019.head(10)

,rok,miasto,nazwa
360,2019,GDYNIA,"Zakup nowoczesnego, W petni wyposazonego ambulansu ratunkowego Z zabudowq medycznq, zgodnego Z wymogami Ministerstwa Zdrowia, dla Miejskiej Stacji Pogotowia Ratunkowego W Gdyni"
361,2019,GDYNIA,"Parki kieszonkowe W 9 dzielnicach: Dqbrowa, Grabówek, Wzgórze Swietego Maksymiliana, Chylonia, Pogórze, Dziatki Lesne, Leszczynki, Pustki Cisowskie, Wielki Kack"
362,2019,GDYNIA,"Ogrodzony wybieg dla psów, W poblizu schroniska dla zwierzgt Ciapkowo"" przy ulicach towicka - Olkuska W dzielnicy Maty Kack, dla psów Z catej Gdyni"
363,2019,GDYNIA,Kampania edukacyjna dot. segregacji odpadów oraz mieciomaty/recyklomaty - urzadzenia do zdawania odpadów aluminiowych i szkta Z wymianq na bilety/znizki Z uzyciem Karty Mieszkanca
364,2019,GDYNIA,"Gdynskie taki kwietne zlokalizowane w 9 dzielnicach: Dqbrowa, Sródmiescie, Karwiny, Obtuze, Cisowa, Chylonia, Wielki Kack, Pogórze oraz Ortowo"
365,2019,GDYNIA,Dostosowanie Estakady Kwiatkowskiego do ruchu rowerowego na odcinku Morska- Hutnicza
366,2019,GDYNIA,Mobilna stacja pomiarowa powietrza i obstugi interwencji zwiqzanych Z porzqdkiem publicznym (dron) oraz kampania informacyjna dot. zanieczyszczania powietrza
367,2019,GDYNIA,Organizacji pórkolonii letnich dla dzieci na terenie 23 szkót podstawowych W 21 dzielnicach Gdyni
368,2019,GDYNIA,"Ogrody deszczowe na petlach autobusowych/trolejbusowych (Chabrowa, Grabówek SKM, Obtuze Maciejewicza, Pogórze (ul. Czernickiego), Dqbrowa Mietowa, Karwiny Tesco, Ortowo SKM Klif)"
369,2019,GDYNIA,Transport i wsparcie dla seniorów (65+) W dotarciu do miejsc uzytecznosci publicznej


In [39]:
df_2019_morf = create_morf_table(df_2019)
df_2019_morf.head()

,orth_,lemma_,tag_,pos_,dep_,head,is_stop,is_punct,is_digit
0,zakup,zakup,SUBST,NOUN,nsubj,kampania,False,False,False
1,nowoczesnego,nowoczesny,ADJ,ADJ,amod,zakup,False,False,False
2,",",",",INTERP,PUNCT,punct,petni,False,True,False
3,w,w,PREP,ADP,case,petni,True,False,False
4,petni,petni,ADJ,NOUN,nmod,zakup,False,False,False


In [40]:
# Usunięcie duplikatów
df_2019_morf = df_2019_morf.drop_duplicates()
df_2019_morf.shape

(85195, 9)

In [41]:
df_2019_morf[ df_2019_morf['pos_']=='VERB']["lemma_"].value_counts().head(30)

mieć            53
ratować         26
bawić           21
uczyć           20
grać            18
chcieć          17
łączyć          15
czytać          13
przejść         13
dbać            12
chronić         12
dać             11
mieszkać        10
poznać          10
ćwiczyć          9
pomóc            9
spotkać          9
pomagać          9
zapraszać        8
wyremontować     7
być              7
zadbać           7
wygon            7
wiedzieć         6
biegać           6
zaklada          6
móc              6
tworzyć          6
zmieniać         6
kochać           5
Name: lemma_, dtype: int64

In [42]:
df_2019_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Spacy_2019.csv', index=False)

## 2020

In [43]:
df_2020 = df[df['rok'] == 2020]
df_2020.head(10)

,rok,miasto,nazwa
0,2020,GDYNIA,Projektistatus
1,2020,GDYNIA,"Doposażenie parku pomiędzy Estakadą Kwiatkowskiego a wieżowcami przy ul. Dantyszka 2,4,8 w siłownię zewnętrzną i elementy małej architektury - pergole, bujane ławki itp."
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy Obłuże rozwijające pamięć i koncentrację
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana"
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy ul. Niskiej na odcinku pomiędzy ul. Bohaterów Getta Warszawskiego i Hodowlaną
5,2020,GDYNIA,Cotygodniowe weekendowe lekcje tańca towarzyskiego dla dorosłych
6,2020,GDYNIA,Meble miejskie zaprojektowane przez dzieci. Podziemia Hal Targowych w Gdyni.
7,2020,GDYNIA,"Odnowienie podwórek w obrębie ulic Płk. Dąbka, Kwiatkowskiego, Sucharskiego, Ledóchowskiego i Maciejewicza"
8,2020,GDYNIA,Poprawa atrakcyjności skweru przy skrzyżowaniu Wielkopolska-Łowicka-Płocka przez nasadzenie roślin ozdobnych
9,2020,GDYNIA,Radarowe wyświetlacze prędkości na ul. Morskiej


In [44]:
df_2020_morf = create_morf_table(df_2020)
df_2020_morf.head()

,orth_,lemma_,tag_,pos_,dep_,head,is_stop,is_punct,is_digit
0,projektistatus,projektistatus,SUBST,PROPN,ROOT,projektistatus,False,False,False
1,doposażenie,doposażenie,GER,NOUN,obj,projektistatus,False,False,False
2,parku,park,SUBST,NOUN,obj,doposażenie,False,False,False
3,pomiędzy,pomiędzy,PREP,ADP,case,estakadą,False,False,False
4,estakadą,estakada,SUBST,NOUN,nmod,doposażenie,False,False,False


In [45]:
# Usunięcie duplikatów
df_2020_morf = df_2020_morf.drop_duplicates()
df_2020_morf.shape

(85075, 9)

In [46]:
df_2020_morf[ df_2020_morf['pos_']=='VERB']["lemma_"].value_counts().head(30)

grać          45
mieć          40
działać       37
stworzyć      23
ratować       20
pomóc         19
przejść       18
chcieć        17
czytać        17
bawić         15
sadzić        15
uczyć         15
zadbać        13
łączyć        13
móc           13
kochać        12
chronić       11
wspierać      10
zakładać      10
dbać          10
pomagać       10
być            9
wychowywać     9
dać            9
obejmować      8
powstać        8
poznać         8
walczyć        7
uratować       7
biegać         7
Name: lemma_, dtype: int64

In [47]:
df_2020_morf.to_csv('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/Spacy_2020.csv', index=False)